In [110]:
########################
### Import Libraries ###
########################

import pandas as pd
import geopandas as gpd
import matplotlib as plt
import numpy as np
import string
from datetime import datetime, timedelta

## Adjacency Matrix of Dutch Municipalities

In [84]:
#####################################
### Load ShapeFile Municipalities ###
#####################################

# Adjacency is based on a ShapeFile from the CBS: https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/wijk-en-buurtkaart-2020

path_cbs_municipality_shapefile = r'C:\Users\matsu\Desktop\Universiteit\MSc Statistics & Data Science\Year 3\Semester 1\Mandatory Courses\Urban Computing\Project\Code\Cloned Repositories\EpiGNN\data\WijkBuurtkaart_2022_v1\gemeente_2022_v1.shp'
# Load the shapefile
gdf = gpd.read_file(path_cbs_municipality_shapefile)

# Remove duplicate municipalities where H2O is included. Also remove the small Belgian enclave in Noord-Brabant.
gdf = gdf[(gdf['H2O'] != 'JA') & (gdf['H2O'] != 'B')]
gdf = gdf.reset_index(drop=True)

In [85]:
###############################
### Create Adjacency Matrix ###
###############################

adjacency_dict = {}

for i, municipality_i in gdf.iterrows():
    adjacent_municipalities = []
    for j, municipality_j in gdf.iterrows():
        if i != j and municipality_i.geometry.touches(municipality_j.geometry):
            adjacent_municipalities.append(municipality_j['GM_NAAM'])
    adjacency_dict[municipality_i['GM_NAAM']] = adjacent_municipalities

# Create a DataFrame to represent the adjacency matrix
adjacency_matrix = pd.DataFrame(index=gdf['GM_NAAM'], columns=gdf['GM_NAAM'], dtype=int).fillna(0)

# Populate the adjacency matrix
for municipality, neighbors in adjacency_dict.items():
    adjacency_matrix.loc[municipality, neighbors] = 1

In [82]:
#############################
### Save Adjacency Matrix ###
#############################

adjacency_matrix.to_csv('adjacency_matrix.csv')

## Dataset COVID-19 Infections per Municipality

In [104]:
####################
### Load Dataset ###
####################

path_covid_daily_infections = r'C:\Users\matsu\Desktop\Universiteit\MSc Statistics & Data Science\Year 3\Semester 1\Mandatory Courses\Urban Computing\Project\Code\Cloned Repositories\EpiGNN\data\COVID-19_aantallen_gemeente_per_dag_tm_03102021.csv'
df_covid_daily_infections_full = pd.read_csv(path_covid_daily_infections, delimiter=';')

In [105]:
#########################################
### Clean COVID-19 Infections Dataset ###
#########################################

# Remove redundant columns
df_covid_daily_infections = df_covid_daily_infections_full[["Date_of_publication", "Municipality_code", "Municipality_name","Province", "Total_reported", "Deceased"]]

# Check for NaNs in Total_reported and Deceased
nan_mask_total_reported = df_covid_daily_infections['Total_reported'].isna()
nan_mask_total_deceased = df_covid_daily_infections['Deceased'].isna()
sum(nan_mask_total_reported), sum(nan_mask_total_deceased)

# Remove those rows for which the Municipality is unknown
df_covid_daily_infections = df_covid_daily_infections.dropna(subset=['Municipality_name'])
nan_mask_total_municipality = df_covid_daily_infections['Municipality_name'].isna()
sum(nan_mask_total_municipality)

# Sorting the DataFrame alphabetically by Municipality name per Day
df_covid_daily_infections = df_covid_daily_infections.sort_values(by=['Date_of_publication', 'Municipality_name'])
df_covid_daily_infections = df_covid_daily_infections.reset_index(drop=True)

# Unique Municipality Names Count (344)
unique_names_count = df_covid_daily_infections['Municipality_name'].nunique()
print(unique_names_count)

344


In [114]:
####################################################
### Convert Dataframe COVID-19 to Desired Format ###
####################################################
# Ensure 'Date_of_publication' is in datetime format
df_covid_daily_infections['Date_of_publication'] = pd.to_datetime(df_covid_daily_infections['Date_of_publication'])

# Date range
start_date = datetime(2020, 2, 28)
end_date = datetime(2021, 10, 3)

# Open the file outside of the loop to avoid reopening it on each iteration
with open('netherlands-covid.txt', 'w') as f:
    # Loop over all dates in the range
    for n in range(int ((end_date - start_date).days)+1):
        date = start_date + timedelta(n)

        # Filter, sort, select
        filtered_df = df_covid_daily_infections[df_covid_daily_infections['Date_of_publication'] == date].sort_values(by='Municipality_name')
        total_reported_values = filtered_df['Total_reported'].tolist()

        # Convert to string and write to file
        f.write(','.join(map(str, total_reported_values)))
        f.write('\n')

**Checking Compatibility of Adjacency Matrix based on ShapeFile 2020 and COVID-19 Dataset**: 

After thoroughly going through the ShapeFile and the COVID-19 dataset manually, we discovered certain mergers of municipalities are not accounted for in the ShapeFile. Although the COVID-19 dataset was collected in 2020, it appears to be using municipalities that weren't approved by Dutch parliament until the beginning of 2022. These mergers are described on the Dutch governmental website. 

**ShapeFile of 2022 Municipalities in the Netherlands**

To remedy this shortcoming we will try using the ShapeFile of 2022. Only the municipality Weesp is non-overlapping, meaning it is in the ShapeFile but not in the dataset. According to the governmental website on COVID-19 datasets, this is because the municipality of Weesp was merged into that of Amsterdam as of March 2022. Despite the data being older, the whole dataset was adjusted to this later change. We will make this change too by adding the respective spots in the adjacency matrix.

In [86]:
##########################################################
### Compare Municipalities Adjacency Matrix & Data set ###
##########################################################

df_covid_daily_infections_unique_municipality_names = set(df_covid_daily_infections['Municipality_name'].unique())

# Solve Merger of Weesp into Amsterdam Municipality 
adjacency_matrix.loc['Amsterdam_mergtemp'] = (adjacency_matrix.loc['Amsterdam'].astype(int) | adjacency_matrix.loc['Weesp'].astype(int)).astype(int)
adjacency_matrix['Amsterdam_mergtemp'] = (adjacency_matrix['Amsterdam'].astype(int) | adjacency_matrix['Weesp'].astype(int)).astype(int)
adjacency_matrix.loc['Amsterdam_mergtemp', 'Amsterdam_mergtemp'] = 0
adjacency_matrix = adjacency_matrix.drop(index=['Amsterdam', 'Weesp'], columns=['Amsterdam', 'Weesp'])

# Rename
new_name = 'Amsterdam'
adjacency_matrix = adjacency_matrix.rename(index={'Amsterdam_mergtemp': new_name})
adjacency_matrix = adjacency_matrix.rename(columns={'Amsterdam_mergtemp': new_name})

# Order alphabetically
adjacency_matrix = adjacency_matrix.sort_index() # Rows
adjacency_matrix = adjacency_matrix.sort_index(axis=1) # Columns

# Find non-overlapping names
adjacency_matrix_unique_municipality_names = set(adjacency_matrix.index)
non_overlapping = adjacency_matrix_unique_municipality_names.symmetric_difference(df_covid_daily_infections_unique_municipality_names)


In [115]:
##########################################
### Save Adjacency Matrix as .txt File ###
##########################################

# Convert the pandas DataFrame to a numpy array
adjacency_matrix_array = adjacency_matrix.to_numpy()

# Save the numpy array to a text file without row and column names
np.savetxt('netherlands-adj.txt', adjacency_matrix_array, fmt='%d', delimiter=',')

In [99]:
################################################
### Simplify Municipality Names for Analysis ###
################################################

# Convert index names to lowercase
adjacency_matrix.index = adjacency_matrix.index.str.lower()

# Replace spaces with underscores
adjacency_matrix.index = adjacency_matrix.index.str.replace(' ', '_')

# Remove punctuation (excluding underscores since they are now part of the names)
translator = str.maketrans('', '', string.punctuation.replace('_', ''))
adjacency_matrix.index = adjacency_matrix.index.str.translate(translator)

# Apply the same transformations to the column names
adjacency_matrix.columns = adjacency_matrix.columns.str.lower()
adjacency_matrix.columns = adjacency_matrix.columns.str.replace(' ', '_')
adjacency_matrix.columns = adjacency_matrix.columns.str.translate(translator)

In [103]:
##################################################
### Create Label File for Dutch Municipalities ###
##################################################

# Create a DataFrame with the municipality names and a range of numbers
output_df = pd.DataFrame({
    'Number': range(1, len(adjacency_matrix.index) + 1),
    'Municipality': adjacency_matrix.index
})

# Format the data as described and save to a .csv file
output_df.to_csv('netherlands-label.csv', header=False, index=False)

with open('netherlands-label.csv', 'w') as f:
    for i, name in enumerate(adjacency_matrix.index, start=1):
        line = f"{name}, {i}"
        f.write(line)
        f.write('\n')


In [ ]:
#################################################
### Create Excel File to Manually Check Names ###
#################################################

adjacency_matrix_unique_municipality_names_list = list(adjacency_matrix_unique_municipality_names)
df_covid_daily_infections_unique_municipality_names_list = list(df_covid_daily_infections_unique_municipality_names)

df_adjacency_names = pd.DataFrame(adjacency_matrix_unique_municipality_names_list, columns=["Adjacency Matrix Names"])
df_dataset_names = pd.DataFrame(df_covid_daily_infections_unique_municipality_names_list, columns=["Dataset COVID-19 Names"])

# Concatenate Dataframes
df_concatenated_municipalities = pd.concat([df_adjacency_names, df_dataset_names], axis = 1)

# Sort each column
df_concatenated_municipalities_sorted = df_concatenated_municipalities.apply(lambda x: x.sort_values().values)

# Save as Excel File
df_concatenated_municipalities_sorted.to_excel('df_concatenated_municipalities_sorted.xlsx', index=False, engine='openpyxl')